In [6]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Logs into MongoDB
username = "aacuser"
password = "password"
ip = "localhost"
port = "46473"
db = "AAC"
shelter = AnimalShelter(username, password, ip, port, db)


# creates a pandas dataframe from cursor generated by readAll function
collection = "animals"
df = pd.DataFrame.from_records(shelter.readAll({}, collection))
df["id"] = df.index


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Shinkle Project Two')

#Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# HTML Layout Code
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={
                    'height' : '150px',
                    'width' : '150px',
                    'float' : 'right',
                    'position' : 'relative',
                    'padding-top' : 0,
                    'padding-right' : 20}
            ),
    html.Center(html.B(html.H1('Michael Shinkle SNHU CS-340 Dashboard'))),
    html.Hr(),
    # code for the interactive filtering options using radio buttons
    dcc.RadioItems(
        id='radioItems',
        options=[
            {'label': 'Water Rescue', 'value': '1'},
            {'label': 'Mountain Rescue', 'value': '2'},
            {'label': 'Disaster Recuse', 'value': '3'},
            {'label': 'Reset', 'value': '4'},
        ],
        value='4',
        labelStyle={'display': 'inline-block'}
        ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns if i!= "id"
        ],
        data=df.to_dict('records'),
        page_action='native',
        page_current=0,
        page_size=10,
        sort_action='native',
        row_selectable='single',
    ),
    html.Br(),
    html.Hr(),
     #This sets up the dashboard so that chart and geolocation map are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('radioItems', 'value')])
def update_dashboard(value):
### mongoDB queries based on value passed from radio buttons
    if value == '1':
        query = {'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
                'sex_upon_outcome': 'Intact Female',
                'age_upon_outcome_in_weeks':{'$gte': 26, '$lte': 156}
                }
        
    elif value == '2':
        query = {'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
                'sex_upon_outcome': 'Intact Male',
                'age_upon_outcome_in_weeks':{'$gte': 26, '$lte': 156}
                }
    elif value == '3':
        query = {'breed': {'$in': ['Doberman Pinscher',' German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
                'sex_upon_outcome': 'Intact Male',
                'age_upon_outcome_in_weeks':{'$gte': 20, '$lte': 300}
                }
    else:
        query = {}
        
    # create new dataframe from cursor returned by query
    df = pd.DataFrame.from_records(shelter.readAll(query, collection))
    df["id"] = df.index
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')


    return (data,columns)

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'derived_viewport_selected_row_ids')]
)
def style_selected_rows(selRow):
    if selRow is None:
        return dash.no_update
    return [
        {"if": {"filter_query": "{{id}} ={}".format(i)}, "backgroundColor": "#DDDDDD",}
        for i in selRow
    ]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # pie chart code   
    dff = pd.DataFrame.from_dict(viewData)
    
    # gets key names and count for pie chart data
    names = list(dff['breed'].value_counts().keys())
    values = list(dff['breed'].value_counts())
    
    fig = px.pie(dff, 
                 values=values, 
                 names=names, 
                 color_discrete_sequence=px.colors.sequential.RdBu,
                 width = 950,
                 height = 500)
    return [dcc.Graph(figure=fig)]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', 'derived_virtual_selected_row_ids')]
    )
def update_map(viewData, selRow):
    dff = pd.DataFrame.from_dict(viewData)
    
    # if no row is selected, defaults map data to the first row of the data table page
    if not selRow:
        selRow = [0]
        
    lat = dff.get('location_lat')[selRow[0]]
    long = dff.get('location_long')[selRow[0]]
    
    # returns map that auto updates based on selected row
    return [
        dl.Map(style={'width': '950px', 'height': '500px'}, 
                center=[lat, long], 
                zoom=10, 
                children=[
                    dl.TileLayer(id="base-layer-id"),
                    # Marker with tool tip and popup
                    dl.Marker(position=[lat, long], children=[
                        dl.Tooltip(dff.get('breed')[selRow[0]]),
                        dl.Popup([
                            html.H3("Animal Name"),
                            html.P(dff.get('name')[selRow[0]])
                        ])
                    ])
                ])
        ]

app

Successfully connected to database AAC
